# Day 2 Solution - Chat Completions API & Ollama Integration

This is my solution to the Day 2 assignment. I've implemented the Chat Completions API with both OpenAI and Ollama.

## Features Implemented:
- Chat Completions API understanding and implementation
- OpenAI API integration with different models
- Ollama local model integration (Llama 3.2)
- Model comparison and testing
- Advanced web scraping with Selenium fallback
- Temperature and token control


In [1]:
# Day 2 Solution - Imports and Setup
import os
import ssl
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from IPython.display import Markdown, display
from openai import OpenAI
from dotenv import load_dotenv
import ollama
import time

# Load environment variables
load_dotenv(override=True)

# SSL fix for Windows
ssl._create_default_https_context = ssl._create_unverified_context
os.environ['PYTHONHTTPSVERIFY'] = '0'
os.environ['CURL_CA_BUNDLE'] = ''

# Initialize OpenAI client
openai = OpenAI()

print("Day 2 setup complete! Ready for Chat Completions API.")


Day 2 setup complete! Ready for Chat Completions API.


In [2]:
# Understanding Chat Completions API
print("## Chat Completions API - Key Concepts")
print("="*50)

print("""
1. **What is Chat Completions API?**
   - The simplest way to call an LLM
   - Takes a conversation and predicts what should come next
   - Invented by OpenAI, now used by everyone

2. **Key Components:**
   - Messages: List of conversation turns
   - Roles: system, user, assistant
   - Models: Different LLMs with different capabilities
   - Parameters: temperature, max_tokens, etc.

3. **Message Format:**
   [
     {"role": "system", "content": "You are a helpful assistant"},
     {"role": "user", "content": "Hello!"},
     {"role": "assistant", "content": "Hi there!"},
     {"role": "user", "content": "What's the weather?"}
   ]
""")

# Test basic Chat Completions
messages = [
    {"role": "system", "content": "You are a helpful programming tutor."},
    {"role": "user", "content": "Explain what a Chat Completions API is in simple terms."}
]

print("\\nTesting basic Chat Completions API...")
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0.7,
    max_tokens=150
)

print(f"Response: {response.choices[0].message.content}")


## Chat Completions API - Key Concepts

1. **What is Chat Completions API?**
   - The simplest way to call an LLM
   - Takes a conversation and predicts what should come next
   - Invented by OpenAI, now used by everyone

2. **Key Components:**
   - Messages: List of conversation turns
   - Roles: system, user, assistant
   - Models: Different LLMs with different capabilities
   - Parameters: temperature, max_tokens, etc.

3. **Message Format:**
   [
     {"role": "system", "content": "You are a helpful assistant"},
     {"role": "user", "content": "Hello!"},
     {"role": "assistant", "content": "Hi there!"},
     {"role": "user", "content": "What's the weather?"}
   ]

\nTesting basic Chat Completions API...
Response: A Chat Completions API is a tool that allows developers to create applications that can interact with users through text-based conversations. Here’s a simple breakdown:

1. **Chat**: This means it can hold a conversation, similar to how you chat with friends or a custom

In [3]:
# Model Comparison - Different OpenAI Models
def test_model(model_name, prompt, temperature=0.7, max_tokens=100):
    """Test different OpenAI models with the same prompt"""
    print(f"\\n🤖 Testing {model_name}...")
    
    messages = [
        {"role": "system", "content": "You are a helpful assistant. Be concise."},
        {"role": "user", "content": prompt}
    ]
    
    try:
        response = openai.chat.completions.create(
            model=model_name,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens
        )
        
        result = response.choices[0].message.content
        print(f"✅ {model_name}: {result}")
        return result
        
    except Exception as e:
        print(f"❌ {model_name}: Error - {e}")
        return None

# Test different models
prompt = "What is machine learning in one sentence?"

models_to_test = [
    "gpt-4o-mini",
    "gpt-4o", 
    "gpt-3.5-turbo"
]

print("## Model Comparison Test")
print("="*50)

results = {}
for model in models_to_test:
    results[model] = test_model(model, prompt)


## Model Comparison Test
\n🤖 Testing gpt-4o-mini...
✅ gpt-4o-mini: Machine learning is a subset of artificial intelligence that enables systems to learn from data and improve their performance over time without being explicitly programmed.
\n🤖 Testing gpt-4o...
✅ gpt-4o: Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed.
\n🤖 Testing gpt-3.5-turbo...
✅ gpt-3.5-turbo: Machine learning is a branch of artificial intelligence that enables computers to learn from data and improve their performance on specific tasks without being explicitly programmed.


In [4]:
# Ollama Integration - Local Model Testing
def test_ollama_model(model_name, prompt):
    """Test Ollama models locally"""
    print(f"\\n🦙 Testing Ollama {model_name}...")
    
    try:
        response = ollama.chat(
            model=model_name,
            messages=[
                {"role": "system", "content": "You are a helpful assistant. Be concise."},
                {"role": "user", "content": prompt}
            ]
        )
        
        result = response['message']['content']
        print(f"✅ Ollama {model_name}: {result}")
        return result
        
    except Exception as e:
        print(f"❌ Ollama {model_name}: Error - {e}")
        return None

# Test Ollama models
print("\\n## Ollama Local Model Testing")
print("="*50)

ollama_models = ["llama3.2", "llama3.2:3b", "llama3.2:1b"]

ollama_results = {}
for model in ollama_models:
    ollama_results[model] = test_ollama_model(model, prompt)


\n## Ollama Local Model Testing
\n🦙 Testing Ollama llama3.2...
✅ Ollama llama3.2: Machine learning is a type of artificial intelligence that enables computers to learn from data, identify patterns, and make predictions or decisions without being explicitly programmed.
\n🦙 Testing Ollama llama3.2:3b...
❌ Ollama llama3.2:3b: Error - model 'llama3.2:3b' not found (status code: 404)
\n🦙 Testing Ollama llama3.2:1b...
❌ Ollama llama3.2:1b: Error - model 'llama3.2:1b' not found (status code: 404)


In [5]:
# Advanced Web Scraping with Selenium Fallback
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def clean_text_from_soup(soup):
    """Extract clean text from BeautifulSoup object"""
    if not soup or not soup.body:
        return ""
    for tag in soup.body(["script", "style", "noscript", "template", "svg", "img", "video", "source", "iframe", "form", "input"]):
        tag.decompose()
    text = soup.body.get_text(separator="\\n", strip=True)
    # Collapse excessive blank lines
    import re
    text = re.sub(r"\\n{3,}", "\\n\\n", text)
    return text

def is_js_heavy(html_text):
    """Check if page needs JavaScript to render content"""
    if not html_text:
        return True
    soup = BeautifulSoup(html_text, "html.parser")
    txt_len = len(re.sub(r"\\s+", " ", soup.get_text()))
    script_tags = html_text.count("<script")
    if txt_len < 1200:  # very little text => likely JS-rendered
        return True
    if script_tags > 50 and (txt_len / (script_tags + 1)) < 40:
        return True
    if re.search(r"(Loading|Please wait|Enable JavaScript)", html_text, re.I):
        return True
    return False

def fetch_static_html(url):
    """Try to fetch HTML using requests (no JS execution)"""
    try:
        r = requests.get(url, headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"}, timeout=15)
        r.raise_for_status()
        return r.text
    except Exception:
        return None

def fetch_js_html(url):
    """Fetch HTML using Selenium (with JS execution)"""
    try:
        options = Options()
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        
        driver.get(url)
        time.sleep(2)  # Wait for JS to execute
        html = driver.page_source
        driver.quit()
        return html
    except Exception as e:
        print(f"JS fetch failed: {e}")
        return None

def fetch_website_contents(url, char_limit=2000, allow_js_fallback=True):
    """Enhanced website content fetching with JS fallback"""
    html = fetch_static_html(url)
    need_js = (html is None) or is_js_heavy(html)

    if need_js and allow_js_fallback:
        html = fetch_js_html(url) or html or ""

    soup = BeautifulSoup(html or "", "html.parser")
    title = soup.title.get_text(strip=True) if soup.title else "No title found"
    text = clean_text_from_soup(soup)
    return (f"{title}\\n\\n{text}").strip()[:char_limit]

print("Advanced web scraping functions defined!")


Advanced web scraping functions defined!


In [6]:
# Model-Agnostic Summarization Function
def summarize_with_model(url, model="gpt-4o-mini", temperature=0.4, max_tokens=None):
    """Summarize website content using any available model"""
    website = fetch_website_contents(url, allow_js_fallback=True)
    
    system_prompt = """
    You are a helpful assistant that analyzes website content
    and provides a clear, concise summary.
    Respond in markdown format. Do not wrap the markdown in a code block.
    """
    
    user_prompt = f"""
    Here are the contents of a website.
    Provide a short summary of this website.
    If it includes news or announcements, then summarize these too.

    {website}
    """
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    try:
        if model.startswith("gpt") or model.startswith("o1"):
            # OpenAI model
            response = openai.chat.completions.create(
                model=model,
                messages=messages,
                temperature=temperature,
                max_tokens=max_tokens
            )
            return response.choices[0].message.content
        else:
            # Ollama model
            response = ollama.chat(
                model=model,
                messages=messages
            )
            return response['message']['content']
    except Exception as e:
        return f"Error with {model}: {e}"

def display_summary_with_model(url, model="gpt-4o-mini", **kwargs):
    """Display website summary using specified model"""
    print(f"🔍 Summarizing {url} with {model}...")
    summary = summarize_with_model(url, model, **kwargs)
    display(Markdown(f"## Summary using {model}\\n\\n{summary}"))

print("Model-agnostic summarization functions defined!")


Model-agnostic summarization functions defined!


In [7]:
# Test Day 2 Solution - Model Comparison
print("## Day 2 Solution Test - Model Comparison")
print("="*60)

# Test with a JavaScript-heavy website
test_url = "https://openai.com"

print(f"Testing website: {test_url}")
print("\\n" + "="*60)

# Test with different models
models_to_test = [
    ("gpt-4o-mini", "OpenAI GPT-4o-mini"),
    ("llama3.2:3b", "Ollama Llama 3.2 3B"),
    ("llama3.2:1b", "Ollama Llama 3.2 1B")
]

for model, description in models_to_test:
    print(f"\\n📊 Testing with {description}...")
    try:
        display_summary_with_model(test_url, model=model, temperature=0.4, max_tokens=200)
    except Exception as e:
        print(f"❌ Error with {description}: {e}")
    
    print("\\n" + "-"*40)


## Day 2 Solution Test - Model Comparison
Testing website: https://openai.com
\n============================================================
\n📊 Testing with OpenAI GPT-4o-mini...
🔍 Summarizing https://openai.com with gpt-4o-mini...
❌ Error with OpenAI GPT-4o-mini: name 're' is not defined
\n----------------------------------------
\n📊 Testing with Ollama Llama 3.2 3B...
🔍 Summarizing https://openai.com with llama3.2:3b...
❌ Error with Ollama Llama 3.2 3B: name 're' is not defined
\n----------------------------------------
\n📊 Testing with Ollama Llama 3.2 1B...
🔍 Summarizing https://openai.com with llama3.2:1b...
❌ Error with Ollama Llama 3.2 1B: name 're' is not defined
\n----------------------------------------
